In [1]:
## Imports

%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
from PIL import Image

import scipy.misc
from skimage import measure   

import time
import os
import zipfile as zf
import tarfile
import csv
import pickle
import urllib
import pandas as pd


from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

# Script to convert Udacity Annotations to Coco Json Annotations
import xml.etree.cElementTree as ET
import argparse
import json
from collections import OrderedDict
from shapely.geometry import Polygon, MultiPolygon, MultiPoint
import ast

/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/pycocotools/coco.py:49: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/dean/miniconda3/envs/ros-kache/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/dean/miniconda3/envs/ros-kache/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/dean/miniconda3/envs/r

In [2]:
## Dataset Parameters ##
WORKING_DIRECTORY = '/media/dean/datastore/ROAD_COCO/darknet/data'
COCO_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'coco')
IMAGES_DIRECTORY = os.path.join(COCO_DIRECTORY, 'images', 'train2014')
img_prefix = 'COCO_train2014_0000'


UNCHANGED_COCO_ANNOTATIONS_FILE = os.path.join(WORKING_DIRECTORY, 'coco','annotations' , 'instances_train2014.json')
FIXED_COCO_ANNOTATIONS_FILE = os.path.join(WORKING_DIRECTORY, 'coco','annotations' , 'fixed_instances_train2014.json')
ROAD_COCO_ANNOTATIONS_FILE = os.path.join(WORKING_DIRECTORY, 'coco','annotations' , 'roadcoco_instances_train2014.json')


COCO_FIXES_CSV = os.path.join(WORKING_DIRECTORY, 'COCO_Trucks.csv')
HEADER_ROW=['ID', 'FileName', 'ImageUrl', 'KEEP']

COCO_TRUCK_CAR_DELETES_CSV = os.path.join(WORKING_DIRECTORY, 'coco_cars_trucks.csv')

## RoadCOCO Dataset Labels
NEW_ROADCOCO_CATEGORIES = os.path.join(WORKING_DIRECTORY,'RoadCOCO_Classes.csv')
RC_HEADER_ROW = ['class', 'super-category', 'special', 'description']

In [3]:
# Get RoadCOCO Labels to Use as Ground Truth# Get Ro 
roadcoco_labels = pd.read_csv(NEW_ROADCOCO_CATEGORIES, names=RC_HEADER_ROW, skiprows=1)
roadcoco_labels.head()

,class,super-category,special,description
0,Vehicle,Vehicle,Orientation matters,Combination of all other vehicles to get a cat...
1,Car,Vehicle,Orientation matters,NaN
2,Pickup,Truck,Orientation matters,NaN
3,TractorSemi,Truck,Orientation matters,NaN
4,TrailerSemi,Truck,Orientation matters,NaN


In [4]:


images_directory = IMAGES_DIRECTORY
road_coco = COCO(UNCHANGED_COCO_ANNOTATIONS_FILE)

categories = road_coco.loadCats(road_coco.getCatIds())
category_names = [category['name'] for category in categories]
print('\nCustom COCO categories: \n{}\n'.format(' '.join(category_names)))

loading annotations into memory...
Done (t=13.43s)
creating index...
index created!

Custom COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush



In [5]:
new_cat_index = 300000
# Represent Category IDs using RoadCOCO Labels
roadcats2cocoids = {}
for i, label in enumerate(sorted(set(roadcoco_labels['class'].tolist()))):
    # if label in coco_categories, use that categoriy id
    if str(label).lower().strip() in category_names:
        cat_id = road_coco.getCatIds(catNms=[str(label).lower().strip()])
        roadcats2cocoids[str(label).lower().strip()] = cat_id[0] if cat_id else i
    else:
        roadcats2cocoids[str(label).lower().strip()] = new_cat_index+i
for i, label in enumerate(sorted(set(roadcoco_labels['super-category'].tolist()))):
    # if label in coco_categories, use that categoriy id
        if str(label).lower().strip() in category_names:
            cat_id = road_coco.getCatIds(catNms=[str(label).lower().strip()])
            roadcats2cocoids[str(label).lower().strip()] = cat_id[0] if cat_id else i
        else:
            roadcats2cocoids[str(label).lower().strip()] = new_cat_index+i
        
cocoids2roadcats = {i: v for v, i in roadcats2cocoids.items()}


# Build Categories List in MS RoadCOCO Format
roadcoco_categories = [] 
for label in roadcoco_labels.as_matrix():
    if label[0].lower().strip() in category_names:
        category = str(label[0]).lower().strip()
        cat_id = roadcats2cocoids[category]
        
        # Verify super-category in list, handle edge cases
        if str(label[1]).lower() == 'truck':
            sup_cat = cocoids2roadcats[roadcats2cocoids['truck']]
        elif str(label[1]).lower() == 'pedestrian':
            sup_cat = cocoids2roadcats[roadcats2cocoids['person']]
        elif str(label[1]).lower() == 'vehicle':
            sup_cat = cocoids2roadcats[roadcats2cocoids['car']]            
    elif label[1].lower().strip() in category_names and label[0].lower().strip() not in category_names :
        category = str(label[1]).lower()
        cat_id = roadcats2cocoids[category]

        # Verify super-category in list, handle edge cases
        if str(label[1]).lower() == 'truck':
            sup_cat = cocoids2roadcats[roadcats2cocoids['truck']]
        elif str(label[1]).lower() == 'pedestrian':
            sup_cat = cocoids2roadcats[roadcats2cocoids['person']]
        elif str(label[1]).lower() == 'vehicle':
            sup_cat = cocoids2roadcats[roadcats2cocoids['car']] 
    else:
        category = str(label[0]).lower().strip()
        cat_id = roadcats2cocoids[category]
        sup_cat = roadcats2cocoids[str(label[1]).lower().strip()]
    
    roadcoco_categories.append({"id": cat_id, "name": category, "supercategory":sup_cat})
        
    
print (len(roadcoco_categories))

65


/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/ipykernel/__main__.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [28]:
## Dataset Parameters ##
images_path = "{}{}".format( IMAGES_DIRECTORY, '*.jpg')
img_prefix = 'COCO_train2014_0000'
    
## VGG Labeler Dataset Extraction Parameters ##
ROADCOCO_DIRECTORY = '/media/dean/datastore/ROAD_COCO/'
UDACITY_DIRECTORY = os.path.join(ROADCOCO_DIRECTORY, 'object-detection-crowdai' )
UDACITY_ANNS_CSV = os.path.join(ROADCOCO_DIRECTORY, 'udacity_detections.csv')
UDACITY_SOURCE_URL = 'http://bit.ly/udacity-annoations-crowdai'
DATASET_ZIPFILE = 'object-detection-crowdai.tar.gz'

HEADER_ROW=['xstart', 'ystart', 'xstop', 'ystop', 'frame', 'label', 'preview_url']


In [34]:
def unzip_file(zip_file, source_dir_name=None, destination=ROADCOCO_DIRECTORY):
    if 'tar.gz' in zip_file:
        head, tail = os.path.splitext(zip_file)
        if not os.path.exists(os.path.join(os.path.splitext(head)[0])):
            print('unzipping file:', zip_file, 'to directory:', os.path.join( os.path.splitext(head)[0]))
            tar = tarfile.open(zip_file, "r:*")
            tar.extractall(destination)
            tar.close()
    else: #.zip extension
        head, tail = os.path.splitext(zip_file)
        if not os.path.exists(os.path.join(destination, head)):
            print('File does not exist: ', os.path.join(destination, head), ': Extracting')
            zipf = zf.ZipFile(os.path.join(WORKING_DIRECTORY,zip_file))
            print('Loaded zipf',zipf, ': Extracting')
            zipf.extractall(os.path.join(destination, head))
            zipf.close()

In [35]:
def maybe_download(source_url, filename):
    if not os.path.exists(IMAGES_DIRECTORY):
        os.mkdir(IMAGES_DIRECTORY)
    
    filepath = os.path.join(IMAGES_DIRECTORY, filename)
    if not os.path.exists(filepath):
        filepath, _ = urllib.request.urlretrieve(source_url, filepath)
        statinfo = os.stat(filepath)
        print('')
        print('Succesfully downloaded:', filepath, '| % d MB.' % int(statinfo.st_size*1e-6))
    return filepath

In [36]:
## Extract Labels ##
annotations = pd.read_csv(UDACITY_ANNS_CSV, names=HEADER_ROW, skiprows=1)
annotations.head()

,xstart,ystart,xstop,ystop,frame,label,preview_url
0,785,533,905,644,1479498371963069978.jpg,Car,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...
1,89,551,291,680,1479498371963069978.jpg,Car,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...
2,268,546,383,650,1479498371963069978.jpg,Car,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...
3,455,522,548,615,1479498371963069978.jpg,Truck,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...
4,548,522,625,605,1479498371963069978.jpg,Truck,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...


In [38]:
## Download Udacity Dataset
tar_file = maybe_download( UDACITY_SOURCE_URL, os.path.join(ROADCOCO_DIRECTORY,DATASET_ZIPFILE))
unzip_file(tar_file)



unzipping file: /media/dean/datastore/ROAD_COCO/object-detection-crowdai.tar.gz to directory: /media/dean/datastore/ROAD_COCO/object-detection-crowdai


In [39]:
urlstofilepaths = {}
url_paths = set(annotations['preview_url'].tolist())
annotations.as_matrix()

for annotation in annotations.as_matrix():
    url = annotation[6]
    fname = annotation[4]
    if url not in urlstofilepaths.keys():
        urlstofilepaths[url] = maybe_download(url, os.path.join(UDACITY_DIRECTORY, fname))
print(urlstofilepaths)

/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


{'http://crowdai.com/images/Wwj-gorOCisE7uxA/visualize': '/media/dean/datastore/ROAD_COCO/object-detection-crowdai/1479498371963069978.jpg', 'http://crowdai.com/images/c3AY-ptjXfT_oC9F/visualize': '/media/dean/datastore/ROAD_COCO/object-detection-crowdai/1479498372942264998.jpg', 'http://crowdai.com/images/cYi8vqAYC3Kgxgey/visualize': '/media/dean/datastore/ROAD_COCO/object-detection-crowdai/1479498373962951201.jpg', 'http://crowdai.com/images/ZEVmHkPemaQHyQvc/visualize': '/media/dean/datastore/ROAD_COCO/object-detection-crowdai/1479498374962942172.jpg', 'http://crowdai.com/images/KgCtg9Fl-JLZeoPW/visualize': '/media/dean/datastore/ROAD_COCO/object-detection-crowdai/1479498375942206592.jpg', 'http://crowdai.com/images/4J54q8HlbmLV3gXs/visualize': '/media/dean/datastore/ROAD_COCO/object-detection-crowdai/1479498373462797835.jpg', 'http://crowdai.com/images/0cQcm8atYFtyRCyP/visualize': '/media/dean/datastore/ROAD_COCO/object-detection-crowdai/1479498376463086347.jpg', 'http://crowdai.com

In [40]:
fixed_coco = COCO(ROAD_COCO_ANNOTATIONS_FILE)

categories = fixed_coco.loadCats(fixed_coco.getCatIds())
category_names = [category['name'] for category in categories]
print('Custom COCO categories: \n{}\n'.format(' '.join(category_names)))

loading annotations into memory...
Done (t=16.68s)
creating index...
index created!
Custom COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush



In [ ]:
images, anns = [], []
    
img_offset = 10000001

num_imgs = len(url_paths)
ann_index = 30000000

 
        
    
for img_id, url in enumerate(sorted(set(url_paths))):
    im = Image.open(urlstofilepaths[url])
    width, height = im.size
    
    if os.path.split(urlstofilepaths[url])[-1].startswith(img_prefix):
        dic = {'file_name': os.path.split(urlstofilepaths[url])[-1], 'id': img_offset+img_id, 'height': height, 'width': width}
    else:
        dic = {'file_name': img_prefix+os.path.split(urlstofilepaths[url])[-1], 'id': img_offset+img_id, 'height': height, 'width': width}
    
    images.append(dic)
    
    for annotation in [x for x in annotations.as_matrix() if x[4].lower() == os.path.split(urlstofilepaths[url])[-1].lower()]:

        
        # xy coords: [xstart, ystart, xstop, ystop] -> bbox = [x,y,width,height]
        xstart, ystart, xstop, ystop = annotation[0], annotation[1], \
                                       annotation[2], annotation[3]
        

        if float(xstart) < 0:
            xstart = 0.0
        if float(ystart) < 0:
            ystart = 0.0
        if float(ystop) <= 0:
            ystop = 3.0

        if float(xstop) <= 0:
            xstop = 3.0


        if float(xstart) > width:
            xstart = width - 3.0
        if float(ystart) > height:
            ystart = height - 3.0
        if float(ystop) > height:
            ystop = height
        if float(xstop) > width:
            xstop = width
            
            
                
        # Get Points from Bounding Box
        pts = []
        pts.append((float(xstart) , float(xstop)))
        pts.append((float(xstop) , float(ystart)))
        pts.append((float(xstop) , float(ystop)))
        pts.append((float(xstart) , float(ystop)))


        # Convert XML Polygon pts to Coco-friendly bounding boxes
        points = MultiPoint(pts)
        xstart,ystart,xstop,ystop = points.bounds

        # Cast to Integers
        xstart,ystart,xstop,ystop = int(xstart),int(ystart),int(xstop),int(ystop)

        binary_mask = np.zeros_like(im)[:,:,0]
        binary_mask[int(xstart):int(xstop), int(ystart):int(ystop)] +=1


        contours = measure.find_contours(binary_mask, 0.5, positive_orientation='low')
        polygons = []
        segmentations = []



        for contour in contours:
            poly = Polygon(contour)
            poly = poly.simplify(1.0, preserve_topology=False)

            if poly and poly.exterior:
                polygons.append(poly)
                segmentation = np.array(poly.exterior.coords).ravel().tolist()
                segmentations.append(segmentation)

        # Combine the polygons to calculate the bounding box and area
        multi_poly = MultiPolygon(polygons)
        if multi_poly.bounds:
            x, y, max_x, max_y = multi_poly.bounds
            width = max_x - x
            height = max_y - y
            bbox = (x, y, width, height)
            area = multi_poly.area


            # Get Label
            cat = annotation[5].lower().strip()

            if cat == 'motorbike':
                category_id = fixed_coco.getCatIds(catNms=['motorcycle'])
            else:
                category_id = fixed_coco.getCatIds(catNms=[cat])

            if not category_id: #Hardcode tv for now
                category_id = fixed_coco.getCatIds(catNms=['tv'])

            if type(category_id) == list:
                category_id = category_id[0]

            annotation = {
                'segmentation': segmentations,
                'iscrowd': 0,
                'image_id': img_offset+img_id, # Don't want to conflict with existing dataset
                'category_id': category_id,
                'id': ann_index,
                'bbox': bbox,
                'area': area
            }
            ann_index+=1
            anns.append(annotation)

/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/ipykernel/__main__.py:22: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [ ]:
print(anns[:10])
print('Total COCO Dataset Images Count:', len(images))
print('Annotations Count:', len(anns))

In [ ]:
from datetime import datetime

INFO = {
    "description": "Road Object-Detections Dataset based on MS COCO",
    "url": "https://kache.ai",
    "version": "0.0.1",
    "year": 2018,
    "contributor": "deanwebb",
    "date_created": datetime.utcnow().isoformat(' ')
}

LICENSES = [
    {
        "id": 1,
        "name": "The MIT License (MIT)",
        "url": "https://opensource.org/licenses/MIT",
        "description":  """
                        The MIT License (MIT)
                        Copyright (c) 2017 Matterport, Inc.

                        Permission is hereby granted, free of charge, to any person obtaining a copy
                        of this software and associated documentation files (the "Software"), to deal
                        in the Software without restriction, including without limitation the rights
                        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
                        copies of the Software, and to permit persons to whom the Software is
                        furnished to do so, subject to the following conditions:

                        The above copyright notice and this permission notice shall be included in
                        all copies or substantial portions of the Software.

                        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
                        IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
                        FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
                        AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
                        LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
                        OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
                        THE SOFTWARE.
                        """
    }
]

coco_output = {'info': INFO, 'licenses': LICENSES, 'images':images, 'annotations':anns, 'categories': categories}
with open(ROAD_COCO_ANNOTATIONS_FILE, 'w') as output_json_file:
    json.dump(coco_output, output_json_file)

In [ ]:
testing_coco = COCO(ROAD_COCO_ANNOTATIONS_FILE)

In [ ]:
testing_coco = COCO(ROAD_COCO_ANNOTATIONS_FILE)
category_ids = testing_coco.getCatIds(catNms=list(category_names))
image_ids = testing_coco.getImgIds()
image_data = testing_coco.loadImgs(image_ids[np.random.randint(0, len(image_ids))])[0]
print(image_data)

In [ ]:
# load and display instance annotations
image = io.imread(os.path.join(IMAGES_DIRECTORY ,image_data['file_name']))
plt.imshow(image); plt.axis('off')
pylab.rcParams['figure.figsize'] = (128.0, 180.0)
annotation_ids = testing_coco.getAnnIds(imgIds=image_data['id'], catIds=category_ids, iscrowd=None)


annotations = testing_coco.loadAnns(annotation_ids)
print(annotations)
testing_coco.showAnns(annotations)

In [ ]:
# Get Dataset Distribution

dataset = {}

for cat in category_ids:
    annotation_ids = testing_coco.getAnnIds(catIds=[cat])
    image_ids = testing_coco.getImgIds(catIds=[cat])
    cat_nm = testing_coco.loadCats(ids=[cat])[0]['name']
    dataset[cat] = (len(annotation_ids), len(image_ids))
    
    print(cat_nm.upper(), '| Annotations:', dataset[cat][0], ' | Images: ',  dataset[cat][1])

